In [ ]:
!wget https://machinehack-be.s3.amazonaws.com/playstore_app_downloads_prediction_weekend_hackathon_16/PlayStoreApps-ParticipantsData.zip

--2020-08-16 20:54:14--  https://machinehack-be.s3.amazonaws.com/playstore_app_downloads_prediction_weekend_hackathon_16/PlayStoreApps-ParticipantsData.zip
Resolving machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)... 52.219.64.120
Connecting to machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)|52.219.64.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 931277 (909K) [application/zip]
Saving to: ‘PlayStoreApps-ParticipantsData.zip’

PlayStoreApps-Parti 100%[===================>] 909.45K   743KB/s    in 1.2s    

2020-08-16 20:54:16 (743 KB/s) - ‘PlayStoreApps-ParticipantsData.zip’ saved [931277/931277]



In [ ]:
!unzip PlayStoreApps-ParticipantsData.zip

Archive:  PlayStoreApps-ParticipantsData.zip
  inflating: PlayStoreApps-ParticipantsData/Train.csv  
  inflating: PlayStoreApps-ParticipantsData/Test.csv  
  inflating: PlayStoreApps-ParticipantsData/Sample_Submission.csv  


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train = pd.read_csv('PlayStoreApps-ParticipantsData/Train.csv')
test = pd.read_csv('PlayStoreApps-ParticipantsData/Test.csv')
sub = pd.read_csv('PlayStoreApps-ParticipantsData/Sample_Submission.csv')

In [ ]:
train.head(5)

,Offered_By,Category,Rating,Reviews,Size,Price,Content_Rating,Last_Updated_On,Release_Version,OS_Version_Required,Downloads
0,ps_id-24654,Finance,4.18,1481,Varies with device,Free,Everyone,May 05 2020,Varies with device,Varies with device,"100,000+"
1,ps_id-35329,Music And Audio,4.81,302,10M,Free,Everyone,Mar 26 2020,3.9.18,4.1 and up,"5,000+"
2,ps_id-11044,Game Casual,4.27,374,27M,Free,Everyone,May 01 2020,1.10.1,4.1 and up,"10,000+"
3,ps_id-36068,Business,4.03,122058,Varies with device,Free,Teen,May 02 2020,Varies with device,Varies with device,"10,000,000+"
4,ps_id-35831,Medical,4.60,358,Varies with device,297.5742,Everyone,Nov 29 2018,Varies with device,Varies with device,"5,000+"


In [ ]:
test.tail(5)

,Offered_By,Category,Rating,Reviews,Size,Price,Content_Rating,Last_Updated_On,Release_Version,OS_Version_Required
24771,ps_id-30176,Shopping,3.74,38,21M,Free,Everyone,Feb 16 2020,2.4.20,4.1 and up
24772,ps_id-5357,Auto And Vehicles,4.80,4,4.5M,Free,Everyone,Jan 02 2019,1.3,5.0 and up
24773,ps_id-1920,Education,4.78,499,1.1M,Free,Everyone,Feb 16 2018,1.1,4.0.3 and up
24774,ps_id-6655,Game Puzzle,4.61,328050,72M,Free,Everyone,May 02 2020,2.5.3,4.1 and up
24775,ps_id-3916,Finance,3.98,1467,18M,Free,Everyone,Apr 30 2020,2.25.2,4.4 and up


In [ ]:
sub.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0


In [ ]:
train.isnull().sum(),test.isnull().sum(),train.shape,test.shape,train.dtypes

(Offered_By             0
 Category               0
 Rating                 0
 Reviews                0
 Size                   0
 Price                  0
 Content_Rating         0
 Last_Updated_On        0
 Release_Version        0
 OS_Version_Required    0
 Downloads              0
 dtype: int64, Offered_By             0
 Category               0
 Rating                 0
 Reviews                0
 Size                   0
 Price                  0
 Content_Rating         0
 Last_Updated_On        0
 Release_Version        0
 OS_Version_Required    0
 dtype: int64, (16516, 11), (24776, 10), Offered_By              object
 Category                object
 Rating                 float64
 Reviews                  int64
 Size                    object
 Price                   object
 Content_Rating          object
 Last_Updated_On         object
 Release_Version         object
 OS_Version_Required     object
 Downloads               object
 dtype: object)

In [ ]:
df=train.append(test,ignore_index=True)

In [ ]:
df['Size_type'] = df['Size'].str.strip().str[-1]
x=[]
for i in df['Size_type']:
  if i=='e':
    x.append(0)
  elif i=='M':
    x.append(1024)
  elif i=='k':
    x.append(1)
df['Size_type']=x
df['size_t']=df.Size.str.extract('(\d+)')
df['size_t']=df['size_t'].fillna(0)
df['Size']=df['Size_type'].astype(int)*df['size_t'].astype(int)
del df['Size_type']
del df['size_t']
df['Size']=df['Size'].replace(0,df['Size'].mean())

In [ ]:
df['Price']=df['Price'].replace('Free',0).astype(float)

In [ ]:
df['Last_Updated_On'] = pd.to_datetime(df['Last_Updated_On'])

In [ ]:
k=['Last_Updated_On']
for i in k:
  df[i+'_'+'year'] = df[i].dt.year
  df[i+'_'+'day'] = df[i].dt.day
  df[i+'_'+'weekofyear'] = df[i].dt.weekofyear
  df[i+'_'+'month'] = df[i].dt.month
  df[i+'_'+'dayofweek'] =df[i].dt.dayofweek
  df[i+'_'+'weekend'] = (df[i].dt.weekday >=5).astype(int)
del df['Last_Updated_On']

In [ ]:
df['Release_Version']=df['Release_Version'].replace('Varies with device','0')

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Release_Version'] = le.fit_transform(df['Release_Version'])
le.classes_

array(['0', '0.0.0-34-animals', '0.0.0-34-leaders', ..., 'Zs 5.2',
       '_18.08.01_ps', '_19.02.01-us_sy_ps'], dtype=object)

In [ ]:
#df = pd.get_dummies(df, columns=['Category','Content_Rating','OS_Version_Required'])
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Category'] = le.fit_transform(df['Category'])
le.classes_
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Content_Rating'] = le.fit_transform(df['Content_Rating'])
le.classes_
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['OS_Version_Required'] = le.fit_transform(df['OS_Version_Required'])
le.classes_

array(['1.0 and up', '1.1 and up', '1.5 and up', '1.6 and up',
       '2.0 and up', '2.0.1 and up', '2.1 and up', '2.2 - 6.0',
       '2.2 and up', '2.3 - 3.2', '2.3 and up', '2.3.3 and up',
       '3.0 and up', '3.1 and up', '3.2 and up', '4.0 - 5.1',
       '4.0 and up', '4.0.3 - 7.1.1', '4.0.3 and up', '4.1 - 7.1.1',
       '4.1 - 8.0', '4.1 and up', '4.2 - 8.0', '4.2 and up', '4.3 and up',
       '4.4 and up', '4.4w and up', '5.0 - 6.0', '5.0 - 8.0',
       '5.0 and up', '5.1 and up', '6.0 and up', '7.0 - 7.1.1',
       '7.0 and up', '7.1 and up', '8.0 and up', 'Varies with device'],
      dtype=object)

In [ ]:
df['Offered_By']=df.Offered_By.str.extract('(\d+)').astype(int)

In [ ]:
train = df[df['Downloads'].isnull()==False]
test = df[df['Downloads'].isnull()==True]
del test['Downloads']

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['Downloads'] = le.fit_transform(train['Downloads'])
le.classes_

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


array(['1,000+', '1,000,000+', '1,000,000,000+', '10+', '10,000+',
       '10,000,000+', '100+', '100,000+', '100,000,000+', '5,000+',
       '5,000,000+', '5,000,000,000+', '50+', '50,000+', '50,000,000+',
       '500+', '500,000+', '500,000,000+'], dtype=object)

In [ ]:
is_hol = train['Downloads'] == 11
df_try = train[is_hol]
train=train.append([df_try],ignore_index=True)

In [ ]:
train_df=train
test_df=test

In [ ]:
X = train_df.drop(labels=['Downloads'], axis=1)
y = train_df['Downloads'].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.10, random_state=101, stratify=y)

In [ ]:
X_train.shape, y_train.shape, X_cv.shape, y_cv.shape

((14865, 15), (14865,), (1652, 15), (1652,))

In [ ]:
from sklearn.metrics import log_loss

In [ ]:
import lightgbm as lgb
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_cv, label=y_cv)

param = {'objective': 'multiclass',
         'num_class': 18,
         'boosting': 'dart',  
         'metric': 'multi_logloss',
         'learning_rate': 0.09, 
         'num_iterations': 400,
         'num_leaves': 31,
         'max_depth': -1,
         'min_data_in_leaf': 30,
         'bagging_fraction':0.9,
         'bagging_freq': 2,
         'feature_fraction': 0.9,
         'lambda_l2': 0.9,
         'min_data_per_group': 500,
         'max_bin': 255,
         'is_unbalance':True
         }

clf = lgb.train(params=param, 
                early_stopping_rounds=200,
                verbose_eval=100,
                train_set=train_data,
                valid_sets=[test_data])

y_pred = clf.predict(X_cv)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/callback.py:189: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')


[100]	valid_0's multi_logloss: 1.20627
[200]	valid_0's multi_logloss: 1.12776
[300]	valid_0's multi_logloss: 1.08776
[400]	valid_0's multi_logloss: 1.08023


In [ ]:
Xtest = test_df

In [ ]:
y_pred = clf.predict(Xtest)

In [ ]:
submission = pd.DataFrame(data=y_pred, columns=sub.columns)
submission=submission*1.05
submission.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
24771,0.362716,0.002456,0.000183,0.000270,0.393142,0.000764,0.005043,0.006406,0.000342,0.252612,0.000930,0.000131,0.000225,0.018401,0.000490,0.002354,0.003291,0.000244
24772,0.477994,0.004988,0.000197,0.037949,0.015959,0.000947,0.227065,0.003875,0.000438,0.135317,0.001073,0.000143,0.018934,0.003894,0.000335,0.118766,0.001909,0.000218
24773,0.004527,0.003906,0.000194,0.000264,0.425327,0.000575,0.000601,0.346631,0.000343,0.023997,0.000789,0.000141,0.000362,0.233470,0.000330,0.000820,0.007517,0.000207
24774,0.000338,0.010160,0.000590,0.000136,0.000493,0.917055,0.000273,0.000826,0.001141,0.000331,0.114237,0.000074,0.000129,0.000737,0.001030,0.000251,0.001733,0.000466
24775,0.000520,0.015244,0.000168,0.000320,0.023737,0.000657,0.000545,0.725340,0.000334,0.001417,0.003754,0.000122,0.000204,0.118656,0.000334,0.000435,0.158021,0.000192


In [ ]:
submission.to_csv('dartre.csv', index=False)

In [ ]:
from google.colab import files
files.download('dartre.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>